# Problem Set #7

## Tc2006 Programming Languages

October 4, 2021.

_Authors of this notebook’s solution:_

- TEAM #
- Student ID and Name:
- Student ID and Name:

**Instructions:**  Modify the Lisp metacircular evaluator code built in class in order to solve the following problems. Make sure all the unit tests are passed.

In [2]:
;; External function required for this notebook.
(require '[clojure.test :refer [is]])

nil

## Main Interpreter Function

### Special Forms in our Lisp Interpreter

- QUOTE: `(QUOTE datum)`
- IF: `(IF condition then-part else-part)`
- LAMBDA: `(LAMBDA (param1 param2 ...) body)`
- LABEL: `(LABEL name expr)`

In [14]:
(declare $eval)

(import 'clojure.lang.IFn)

(defrecord Closure [env params body]
  IFn
  (applyTo [self args]
    (let [extended-env (merge @env (zipmap params args))]
      ($eval body extended-env))))

(defn make-closure
  [env params body]
  (->Closure (atom env) params body))

(defn set-closure-env!
  [closure label]
  (swap! (.env closure)
         #(assoc % label closure)))

(defn $eval-variable
  [expr env]
  (if (contains? env expr)
    (env expr)
    (throw (RuntimeException. 
            (str "Variable " expr " not found in current scope!")))))

(defn $eval-if
  [expr env]
  (let [[_ condition then-part else-part] expr] 
    (if ($eval condition env)
      ($eval then-part env)
      ($eval else-part env))))

(defn $eval-fun-apply
  [expr env]
  (let [[fun & args] (map #($eval % env) expr)]
    (apply fun args)))

(defn $eval-lambda
  [expr env]
  (let [[_ params body] expr]
    (make-closure env params body)))

(defn $eval-label
  [expr env]
  (let [[_ label lambda] expr
        closure ($eval lambda env)]
    (set-closure-env! closure label)
    closure))

(defn $eval-do
  [expr env]
  (loop [exprs (rest expr)
         last-result nil]
    (if (empty? exprs)
      last-result
      (recur (rest exprs)
             ($eval (first exprs) env)))))

(defn $eval-let
  [expr env]
  (let [[_ [variable var-expr] body] expr
        value ($eval var-expr env)
        extended-env (assoc env variable value)]
    ($eval body extended-env)))
      
(defn $eval-list
  [expr env]
  (case (first expr)
    nil ()
    QUOTE (second expr)
    IF ($eval-if expr env)
    LAMBDA ($eval-lambda expr env)
    LABEL ($eval-label expr env)
    DO ($eval-do expr env)
    LET ($eval-let expr env)
    ($eval-fun-apply expr env)))

(defn $eval
  [expr env]
  (cond
    (symbol? expr) ($eval-variable expr env)
    (list? expr) ($eval-list expr env)
    :else expr))

#'user/$eval

## Unit Tests

In [4]:
;; Test that ordinary values evaluate to themselves.
(is (= 42 ($eval 42 {})))
(is (= true ($eval true {})))
(is (= "hello" ($eval "hello" {})))
(is (= nil ($eval nil {})))

true

In [5]:
;; Test that symbols represent bindings in the given scope (env).
(is (= 7 ($eval 'x '{x 7})))
(is (= 10 ($eval 'y '{x 7, y 10})))
(try
  ($eval 'z '{x 7, y 10})
  (is false)
  (catch RuntimeException e (is true)))

true

In [6]:
;;; Unit tests for QUOTE special form
(is (= 'x ($eval '(QUOTE x) '{x 5})))
(is (= '(+ 2 3) ($eval '(QUOTE (+ 2 3)) {})))
(is (= 42 ($eval '(QUOTE 42) {})))

true

In [7]:
;;; Unit tests for IF special form
(is (= 2 ($eval '(IF 1 2 3) {})))
(is (= 3 ($eval '(IF nil 2 3) {})))
(is (= 10 ($eval '(IF x y z) '{x 5, y 10, z 20})))
(is (= 20 ($eval '(IF x y z) '{x false, y 10, z 20})))

true

In [8]:
;;; Unit tests for function application
(is (= 1 ($eval '(CAR (QUOTE (1 2 3))) {'CAR first})))
(is (= 'B ($eval '(CAR (CDR (QUOTE (A B C D E)))) {'CAR first, 'CDR rest})))
(is (= 16 ($eval '(+ (* 3 2) 5 (- 10 5)) {'+ +, '* *, '- -})))

true

In [9]:
;;; Unit tests for the make-closure function
(def c (make-closure {'+ +} '(x) '(+ x 1)))
(is (= {'+ +} @(.env c)))
(is (= '(x) (.params c)))
(is (= '(+ x 1) (.body c)))
(is (= 6 (apply c '(5))))

true

In [10]:
;;; Unit tests for the LAMBDA special form
(is (= 6 ($eval '((LAMBDA (X) (PLUS X 1)) 5) {'PLUS +})))
(is (= 15 ($eval '((LAMBDA (X Y)
                    (* X (+ Y 1)))
                  3
                  4)
                {'+ +, '* *})))
(is (= 42 ($eval '((LAMBDA () 42)) {})))
(is (= 8 ($eval '(((LAMBDA (X)
                     (LAMBDA (Y) 
                       (+ X Y)))
                   5)
                  3)
                {'+ +})))
(is (= 8 ($eval '((LAMBDA (F X) 
                    (F (F (F X))))
                  (LAMBDA (X) (* X 2))
                  1)
                {'* *})))

true

In [11]:
;;; Unit tests for the LABEL special form
(is (= '(A A B B C C D D) 
       ($eval '((LABEL DUP (LAMBDA (X)
                             (IF (NULL? X)
                               ()
                               (CONS (CAR X)
                                     (CONS (CAR X)
                                           (DUP (CDR X)))))))
                (QUOTE (A B C D)))
              {'NULL? empty?
               'CONS cons
               'CAR first
               'CDR rest})))
(is (= 120 ($eval '((LABEL FACT (LAMBDA (N)
                                  (IF (ZERO? N)
                                    1
                                    (* N (FACT (- N 1))))))
                    5)
                  {'* *
                   'ZERO? zero?
                   '- -})))

true

---

## Problem 1

Add to the Lisp interpreter the `DO` special form, which has the following syntax: 

$$
\texttt{(DO } \mathit{exp}_1 \; \mathit{exp}_2 \; \cdots \; \mathit{exp}_n \texttt{)}
$$

where every $\mathit{exp}_i$ is an expression. This construct behaves like its Clojure counterpart of the same name: it evaluates each expression in order from left to right (most likely for their side effects, such as printing), and returns the result of $\mathit{exp}_n$ (the last expression) or `nil` if there are no expressions.

Example:

    ($eval '(DO (PRN (+ -2 (+ 1 2)))
                (PRN (+ 1 1))
                (PRN 3)
                (+ 2 2))
           {'+ +
            'PRN prn})
    => 4
   
Output:

    1
    2
    3

In [13]:
;;; Unit tests:
(is (nil? ($eval '(DO)
                 {})))
(is (= 3
       ($eval '(DO (+ 1 2))
              {'+ +})))
(is (= 10
       ($eval '(DO (+ 2 5)
                   (- 2 5)
                   (/ 2 5)
                   (* 2 5))
              {'+ +
               '- -
               '/ /
               '* *})))
(is (= "7-32/510"
       (with-out-str
         ($eval '(DO (PR (+ 2 5))
                     (PR (- 2 5))
                     (PR (/ 2 5))
                     (PR (* 2 5)))
                {'PR pr
                 '+ +
                 '- -
                 '/ /
                 '* *}))))
(is (= (let [nl (System/lineSeparator)]
         (str "1" nl "2" nl "3" nl))
       (with-out-str
         ($eval '(DO (PRN (+ -2 (+ 1 2)))
                     (PRN (+ 1 1))
                     (PRN 3)
                     (+ 2 2))
                {'+ +
                 'PRN prn}))))
(is (= (let [nl (System/lineSeparator)]
         (str "One potato," nl
              "two potatoes," nl
              "three potatoes," nl
              "four." nl))
       (with-out-str
         ($eval '(DO (PRINTLN "One potato,")
                     (PRINTLN "two potatoes,")
                     (PRINTLN "three potatoes,")
                     (PRINTLN "four."))
                {'PRINTLN println}))))

true

---

## Problem 2

Add to the Lisp interpreter the `LET` special form, which has the following syntax: 

$$
\texttt{(LET}\texttt{ (}\mathit{var}\texttt{ }\mathit{expr}\texttt{) } \mathit{body}\texttt{)}
$$

where $\mathit{var}$ is a symbol, while $\mathit{expr}$ and $\mathit{body}$ are expressions. This construct evaluates and returns the result of $\mathit{body}$ using an extended environment where $\mathit{var}$ is bound with the result of evaluating $\mathit{expr}$. In other words, it's equivalent to:

$$
\texttt{((LAMBDA}\texttt{ (}\mathit{var}\texttt{) }\mathit{body}\texttt{) } \mathit{expr}\texttt{)}
$$

Examples:

    ($eval '(LET (x 6)
              (* 7 x))
           {'* *})
    => 42

    ($eval '(LET (x (* 2 5))
              (LET (y (+ 1 x))
                (+ 1 (* y x))))
           {'+ +
            '* *})
    => 111

In [15]:
;;; Unit tests:
(is (= 7
      ($eval '(LET (a 7) a) {})))
(is (= 42
      ($eval '(LET (x 6)
                (* 7 x))
        {'* *})))
(is (= 111
      ($eval '(LET (x (* 2 5))
                (LET (y (+ 1 x))
                  (+ 1 (* y x))))
        {'+ +
         '* *})))
(is (= 60
      ($eval '((LAMBDA (x y)
                 (LET (y (+ 1 y))
                   (* x y)))
                10
                5)
        {'* *
         '+ +})))
(is (= '(a b c d)
      ($eval '(LET (one (QUOTE (c d)))
                (LET (two (CONS (QUOTE b) one))
                  (LET (three (CONS (QUOTE a) two))
                    three)))
        {'CONS cons})))
(is (= 8
      ($eval '(((LAMBDA (x y)
                  (LET (z (LET (r) (* x y)) r)
                    (LAMBDA (w)
                      (LET (t (+ x (LET (a y) a) z w))
                        (LET (t t) t))))) 1 2) 3)
        {'* *
         '+ +})))

true

---

## Problem 3

Add to the Lisp interpreter the `DOTIMES` special form, which has the following syntax: 

$$
\texttt{(DOTIMES}\texttt{ (}\mathit{var}\texttt{ }\mathit{count}\texttt{) } \mathit{body}\texttt{)}
$$

where $\mathit{var}$ is a symbol, while $\mathit{count}$ and $\mathit{body}$ are expressions. This construct executes $\mathit{body}$ (which must perform some side effect operation, typically printing) once for each integer from 0 (inclusive) to $\mathit{count}$ (exclusive), while binding the variable $\mathit{var}$ to the integer for the current iteration. This special form always returns `nil`. 

Example:

    ($eval '(DOTIMES (i (+ 2 2))
              (PRINTLN "Line" i))
           {'PRINTLN println, '+ +})
    
    ($eval '(DOTIMES (x 10)
              (PR x))
           {'PR pr})
   
Output:

    Line 0
    Line 1
    Line 2
    Line 3

    0123456789

In [ ]:
;;; Unit tests:
(is (= ""
       (with-out-str ($eval '(DOTIMES (x 0)
                               (PRINTLN x))
                            {'PRINTLN println}))))
(is (= "0123456789"
       (with-out-str ($eval '(DOTIMES (x 10)
                               (PR x))
                            {'PR pr}))))
(is (= (let [nl (System/lineSeparator)]
         (str "Line 0" nl "Line 1" nl "Line 2" nl "Line 3" nl))
       (with-out-str ($eval '(DOTIMES (i (+ 2 2))
                               (PRINTLN "Line" i))
                            {'PRINTLN println, '+ +}))))
(is (= "1-4-9-16-25-36-49-64-81-100-"
       (with-out-str ($eval '(DOTIMES (some-var (* 2 5))
                               (PRINTF "%d-"
                                       ((LAMBDA (x) (* x x))
                                        (INC some-var))))
                            {'PRINTF printf, '* *, 'INC inc}))))
(is (= (str "**************************************************"
            "**************************************************")
       (with-out-str ($eval '(DOTIMES (mxyzptlk (* 2 2 5 5))
                               (PRINT "*"))
                            {'PRINT print, '* *}))))

---

## Problem 4

Add to the Lisp interpreter the `COND` special form, which has the following syntax: 

$$
\texttt{(COND }
  \mathit{test}_1 \; \mathit{exp}_1 \;
  \mathit{test}_2 \; \mathit{exp}_2 \;
  \cdots \;
  \mathit{test}_n \; \mathit{exp}_n
  \texttt{)}
$$

where every $\mathit{test}_i$ and $\mathit{exp}_i$ is an expression. This construct, like its Clojure counterpart of the same name, takes a sequence of $\mathit{test}$/$\mathit{exp}$ pairs. It evaluates each $\mathit{test}$ one at a time. If a $\mathit{test}$ returns logical true (any value not equal to `nil` nor `false`), `COND` evaluates and returns the value of the corresponding $\mathit{exp}$ and doesn't evaluate any of the other $\mathit{test}$s or $\mathit{exp}$s. If there are no tests that evaluate to true, `COND` returns `nil`.

You can alternatively think of `COND` as being equivalent to: 

$$
\texttt{(if } \mathit{test}_1 \; \mathit{exp}_1 \;
\texttt{(if } \mathit{test}_2 \; \mathit{exp}_2 \;
\cdots \;
\texttt{(if } \mathit{text}_n \; \mathit{exp}_n \; \texttt{nil)}
\cdots
\texttt{))}
$$

Example:

    ($eval '(COND
              (= x 1) (QUOTE one)
              (= x 2) (QUOTE two)
              (= x 3) (QUOTE three)
              (= x 4) (QUOTE four)
              true    (QUOTE other))
              {'x 3
               '= =})
    => three

In [ ]:
;;; Unit tests:
(is (nil? ($eval '(COND)
                 {})))
(is (= 2
       ($eval '(COND
                 1 2)
              {})))
(is (nil? ($eval '(COND
                    false 2)
                 {})))
(is (= 'three
       ($eval '(COND
                 (= x 1) (QUOTE one)
                 (= x 2) (QUOTE two)
                 (= x 3) (QUOTE three)
                 (= x 4) (QUOTE four)
                 true    (QUOTE other))
              {'x 3
               '= =})))
(is (= 'other
       ($eval '(COND
                 (= x 1) (QUOTE one)
                 (= x 2) (QUOTE two)
                 (= x 3) (QUOTE three)
                 (= x 4) (QUOTE four)
                 true    (QUOTE other))
              {'x 5
               '= =})))
(is (nil? ($eval '(COND
                    (= x 1) (QUOTE one)
                    (= x 2) (QUOTE two)
                    (= x 3) (QUOTE three)
                    (= x 4) (QUOTE four))
                 {'x 5
                  '= =})))
(is (= "Lannister"
       (with-out-str
         ($eval '(COND
                   (< 4 got) (PRINT "Targaryen")
                   (< 3 got) (PRINT "Stark")
                   (< 2 got) (PRINT "Tully")
                   (< 1 got) (PRINT "Lannister")
                   true      (PRINT "Tyrell"))
                {'got 2
                 '< <
                 'PRINT print}))))